In [50]:
import numpy as np
import pandas as pd


from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import set_config

In [51]:
df = pd.read_csv('E:\\Datasets\\train - train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [52]:
df = df.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [53]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [54]:
x = df.drop(columns = ['Survived'])

y = df['Survived']



In [55]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.2,random_state = 2)

In [56]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
30,1,male,40.0,0,0,27.7208,C
10,3,female,4.0,1,1,16.7000,S
873,3,male,47.0,0,0,9.0000,S
182,3,male,9.0,4,2,31.3875,S
876,3,male,20.0,0,0,9.8458,S


In [57]:
numerical_columns = ['Age','Fare']

numeric_pipeline = Pipeline(steps=[
    
    ('imputer',SimpleImputer(strategy = 'median')),
    
    ('scaler',StandardScaler())
    
])

categorical_columns = ['Embarked','Sex']

categorical_pipeline = Pipeline(steps=[
    
    ('imputer',SimpleImputer(strategy = 'most_frequent')),
    
    ('ohe',OneHotEncoder(handle_unknown = 'ignore'))
    
    
])

In [58]:
feature_transformer = ColumnTransformer(transformers = [
    
    ('num',numeric_pipeline,numerical_columns),
    
    ('cat',categorical_pipeline,categorical_columns)
    
])

In [59]:
clf = Pipeline(steps = [
    
    ('prepocessor',feature_transformer),
    
    ('classifier',LogisticRegression())
    
])

In [60]:
set_config(display='diagram')
clf

Pipeline(steps=[('prepocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Embarked', 'Sex'])])),
                ('classifier', LogisticRegression())])

In [83]:
param_grid = {
    
   'prepocessor__num__imputer__strategy':['mean','mode'],
    
    'prepocessor__cat__imputer__strategy':['constant','most_frequent'],
    
    'classifier__C' :  [.1,1,10,50,100]
    
}

grid_search = GridSearchCV(clf,param_grid,cv = 10)

In [84]:
grid_search.fit(x_train,y_train)

C:\Users\Moavia computer\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Moavia computer\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Moavia computer\anaconda3\lib\site-packages\sklearn\pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\Moavia computer\anaconda3\lib\site-packages\sklearn\pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('prepocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Age',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Embarked',
                                                                          'Sex'])])),
                                       ('classifier', LogisticRegression())]),
             param_grid={'classifier__C': [0.1, 1, 10, 50, 100],
                         'prepocessor__cat__imputer__strategy': ['constant',
                                                                 'most_frequent'],
                         'prepocessor__num__imputer__strategy': ['mean',
                                                                 'mode']})

In [88]:
print(grid_search.best_params_)

{'classifier__C': 1, 'prepocessor__cat__imputer__strategy': 'constant', 'prepocessor__num__imputer__strategy': 'mean'}


In [96]:
print(f'cross vadiation score:{grid_search.best_score_ :.3f}')

cross vadiation score:0.788


In [101]:
print(grid_search.cv_results_)

{'mean_fit_time': array([0.018226  , 0.00156229, 0.01873748, 0.00602069, 0.02492886,
       0.00642574, 0.02454705, 0.00160549, 0.02631834, 0.00159874,
       0.02428753, 0.00256581, 0.01807153, 0.00321178, 0.01542587,
       0.00099721, 0.01658835, 0.00312448, 0.0106472 , 0.00040255]), 'std_fit_time': array([0.00906344, 0.00468686, 0.00475198, 0.0040146 , 0.00452161,
       0.00428884, 0.00440968, 0.00321097, 0.00470822, 0.00319748,
       0.00665052, 0.00496629, 0.00581523, 0.00393362, 0.00334406,
       0.00299163, 0.00251433, 0.00624895, 0.0009255 , 0.00080509]), 'mean_score_time': array([0.00944259, 0.        , 0.00462196, 0.        , 0.00804641,
       0.        , 0.00843389, 0.        , 0.00823572, 0.        ,
       0.00503099, 0.        , 0.00582464, 0.        , 0.00364907,
       0.        , 0.00378354, 0.        , 0.00160358, 0.        ]), 'std_score_time': array([0.00679752, 0.        , 0.00449338, 0.        , 0.00310945,
       0.        , 0.00295393, 0.        , 0.0031663

In [106]:
validation_result = pd.DataFrame(grid_search.cv_results_)
validation_result.sort_values('mean_test_score',ascending=False)
validation_result[['param_classifier__C','param_prepocessor__cat__imputer__strategy','param_prepocessor__num__imputer__strategy','mean_test_score']]

,param_classifier__C,param_prepocessor__cat__imputer__strategy,param_prepocessor__num__imputer__strategy,mean_test_score
0,0.1,constant,mean,0.786444
1,0.1,constant,mode,NaN
2,0.1,most_frequent,mean,0.786444
3,0.1,most_frequent,mode,NaN
4,1,constant,mean,0.787852
5,1,constant,mode,NaN
6,1,most_frequent,mean,0.787852
7,1,most_frequent,mode,NaN
8,10,constant,mean,0.787852
9,10,constant,mode,NaN
